In [12]:
import torch
from diffusers import AnimateDiffPipeline, LCMScheduler, MotionAdapter
from diffusers.utils import export_to_gif, export_to_video, load_image

In [2]:
adapter = MotionAdapter.from_pretrained("wangfuyun/AnimateLCM", torch_dtypr=torch.float16)
pipe = AnimateDiffPipeline.from_pretrained("emilianJR/epiCRealism", motion_adapter=adapter, torch_dtype=torch.float16)
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config, beta_schedule="linear")

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

/home/anaconda3/envs/diffusers/lib/python3.10/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
The config attributes {'center_input_sample': False, 'flip_sin_to_cos': True, 'freq_shift': 0, 'mid_block_type': 'UNetMidBlock2DCrossAttn', 'only_cross_attention': False, 'attention_head_dim': 8, 'dual_cross_attention': False, 'class_embed_type': None, 'addition_embed_type': None, 'num_class_embeds': None, 'upcast_attention': False, 'resnet_time_scale_shift': 'default', 'resnet_skip_time_act': False, 'resnet_out_scale_factor': 1.0, 'time_embedding_type': 'positional', 'time_embedding_dim': None, 'time_embedding_act_fn': None, 'timestep_post_act': None, 'conv_in_kernel': 3, 'conv_out_kernel': 3, 'projection_class_embeddings_input_dim': None, 'class_embeddings_concat': False, 'mid_block_only_cross_attention': N

In [3]:
torch_device = 'cuda'

adapter.to(torch_device)
pipe.to(torch_device)

AnimateDiffPipeline {
  "_class_name": "AnimateDiffPipeline",
  "_diffusers_version": "0.27.2",
  "_name_or_path": "emilianJR/epiCRealism",
  "feature_extractor": [
    "transformers",
    "CLIPFeatureExtractor"
  ],
  "image_encoder": [
    null,
    null
  ],
  "motion_adapter": [
    "diffusers",
    "MotionAdapter"
  ],
  "scheduler": [
    "diffusers",
    "LCMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNetMotionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [4]:
pipe.load_lora_weights("wangfuyun/AnimateLCM", weight_name ="AnimateLCM_sd15_t2v_lora.safetensors", adapter_name="lcm-lora")
pipe.set_adapters(["lcm-lora"], [0.8])
pipe.enable_vae_slicing()
# pipe.enable_model_cpu_offload()

AnimateLCM_sd15_t2v_lora.safetensors:   0%|          | 0.00/135M [00:00<?, ?B/s]

# text to video

In [6]:
output = pipe(
    prompt="A space rocket with trails of smoke behind it launching into space from the desert, 4k, high resolution",
    negative_prompt="bad quality, worse quality, low resolution",
    num_frames=16,
    guidance_scale=2.0,
    num_inference_steps=6,
)

  0%|          | 0/6 [00:00<?, ?it/s]

In [10]:
frames = output.frames[0]
export_to_gif(frames, "./samples/AnimateLCM/t2v/animatelcm.gif")

'./samples/AnimateLCM/t2v/animatelcm.gif'

# image to video

In [38]:
image = load_image("https://huggingface.co/datasets/diffusers/docs-images/resolve/main/i2vgen_xl_images/img_0009.png")
image = load_image(image).convert("RGB")

In [40]:
output = pipe(
    prompt="Papers were floating in the air on a table in the library",
    negative_prompt="bad quality, worse quality, low resolution, ugly face, v",
    image=image, num_frames=16, guidance_scale= 10, num_inference_steps=30,
    generator = torch.manual_seed(42)
).frames[0]

  0%|          | 0/30 [00:00<?, ?it/s]

In [41]:
export_to_gif(output, "./samples/AnimateLCM/i2v/paper.gif", fps=7)

'./samples/AnimateLCM/i2v/paper.gif'